In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_validate, GridSearchCV

In [11]:
df = pd.read_csv('demo_sales.csv')

### Create features

In [ ]:
target_map = data['sales'].to_dict()

data['lag1'] = (data['date'] - pd.Timedelta('364 days')).map(target_map)
data['lag2'] = (data['date'] - pd.Timedelta('728 days')).map(target_map)
data['lag3'] = (data['date'] - pd.Timedelta('1092 days')).map(target_map)

In [20]:
df['Date'] = df['Date'].str.replace('/', '-').head()
pd.to_datetime(df['Date'], errors='ignore', dayfirst=True)

C:\Users\Vector\AppData\Local\Temp\ipykernel_24460\2638803245.py:2: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  pd.to_datetime(df['Date'], errors='ignore', dayfirst=True)


0      2020-01-01
1      2020-01-02
2      2020-01-03
3      2020-01-04
4      2020-01-05
          ...    
1089          NaT
1090          NaT
1091          NaT
1092          NaT
1093          NaT
Name: Date, Length: 1094, dtype: datetime64[ns]

In [21]:
#df.iloc[271]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1094 entries, 0 to 1093
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    5 non-null      object 
 1   Units   1094 non-null   int64  
 2   Sales   1094 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 25.8+ KB


In [ ]:
X = df.drop(['Sales', 'Date'], axis=1)
y = df['Sales']

### Preprocess

In [ ]:

num_features = X

preprocessor = ColumnTransformer(
    [
        ('StandardScaler', StandardScaler(), num_features),
    ]
) 

In [ ]:
X = preprocessor.fit_transform()

In [ ]:
# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_side=0.2, random_state=1, shuffle=False)

### Evaluate models

In [ ]:
def evaluate_model(y_true, y_pred):
    rmse = root_mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return rmse, r2

NameError: name 'LinearRegression' is not defined

In [ ]:
models = {
    'LinearRegression': LinearRegression(),
    'Lasso': Lasso(),
    'Ridge': Ridge(),
    'XGBRegressor': XGBRegressor()
}

model_list = []
r2_list = []
rmse_list = []
results = []
param_grid = {'': [0, 1]}


for model_name, model in models.items():
    time_series_split = TimeSeriesSplit(n_splits=6)
    cv_scores = cross_validate(model, X, y, cv=time_series_split)
    #cv_results = cross_val_score(model, X_train, y_train, cv=cv)
    
    # Fine tune hyperparams
    gs = GridSearchCV(model, cv=time_series_split, param_grid=param_grid)
    gs.fit(X, y)
    results.append(cv_scores['test_score'].mean())

    #train_rmse, train_r2 = evaluate_model() # ???
    #y_pred_test = model.predict(X_test)
    #test_rmse, test_r2 = evaluate_model(y_test, y_pred_test)
    print(model_name)
    print('Model performance for Training set')
    print(f'- RMSE: {train_rmse}')
    print(f'- R2: {train_r2}')
    print('-----------------------------')
    print('Model performance for Test set')
    print(f'- RMSE: {test_rmse}')
    print(f'- R2: {test_r2}')

plt.boxplot(results, labels=models.keys())
plt.show()

In [ ]:
para = param[LinearRegression()]
# or this, not sure 
# para = param['LinearRegression']

# Perform randomized grid search on train data and find best params
gs = RandomizedSearchCV(model, para, cv=time_series_split)
gs.fit(X, y)
model.set_params(**gs.best_params_)
gs.best_score_
cv_results_

time_series_split = TimeSeriesSplit(n_splits=6, test_size=7)
cv_scores = cross_validate(model, X, y, cv=time_series_split)
print("CV score: ", cv_scores['test_score'].mean())